In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from keras.datasets import mnist

from keras.layers import *
from keras.models import Sequential, Model
from keras.optimizers import Adam
import keras

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
(X_train , _),(_,_)=mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
X_train.shape

(60000, 28, 28)

In [ ]:
X_train=(X_train-127.5)/127.5
print(X_train.min())
print(X_train.max())

-1.007843137254902
-0.9921568627450981


In [ ]:
TOTAL_EPOCHS = 50
BATCH_SIZE = 256
HALF_BATCH = 128

NO_OF_BATCHES = int(X_train.shape[0] / BATCH_SIZE)

NOISE_DIM = 100

adam = Adam(learning_rate=2e-4, beta_1=0.5)

In [ ]:
# Generator Model : Upsampling

generator = Sequential()
generator.add(Dense(units=7*7*128, input_shape=(NOISE_DIM,)))
generator.add(Reshape((7, 7, 128)))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())

# (7,7,128) -> (14,14,64)
generator.add(Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same'))
generator.add(Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same'))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())

# (14,14,64) --> (28,28,1)
generator.add(Conv2DTranspose(1, (3, 3), strides=(1, 1), padding='same', activation='tanh'))
generator.compile(loss=keras.losses.BinaryCrossentropy(), optimizer=adam)

generator.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 6272)           │       633,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_3              │ (None, 14, 14, 64)     │        73,792 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_4              │ (None, 28, 28, 64)     │        36,928 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_6 (LeakyReLU)       │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 28, 28, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_5              │ (None, 28, 28, 1)      │           577 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 745,537 (2.84 MB)

 Trainable params: 745,153 (2.84 MB)

 Non-trainable params: 384 (1.50 KB)

In [ ]:
# Discriminator Model -> i.e. Down Sampling
# (28,28,1) -> (14,14,64)

discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=(3, 3), strides=(2, 2), padding='same', input_shape=(28, 28, 1)))
discriminator.add(LeakyReLU(0.2))

# (14,14,64) --> (7,7,128)
discriminator.add(Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding='same'))

discriminator.add(LeakyReLU(0.2))

# (7,7,128) --> 6272
discriminator.add(Flatten())
discriminator.add(Dense(100))
discriminator.add(LeakyReLU(0.2))

discriminator.add(Dense(1, activation='sigmoid'))

discriminator.compile(loss=keras.losses.BinaryCrossentropy(), optimizer=adam)

discriminator.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 14, 14, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 7, 7, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │       627,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 701,897 (2.68 MB)

 Trainable params: 701,897 (2.68 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Combined Model

discriminator.trainable = False

gan_input = Input(shape=(NOISE_DIM,))

generated_img = generator(gan_input)

gan_output = discriminator(generated_img)

# Functional API
model = Model(gan_input, gan_output)

model.compile(loss=keras.losses.binary_crossentropy, optimizer=adam)

In [ ]:
model.summary()

Model: "functional_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (None, 28, 28, 1)      │       745,537 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 1)              │       701,897 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,447,434 (5.52 MB)

 Trainable params: 745,153 (2.84 MB)

 Non-trainable params: 702,281 (2.68 MB)

In [ ]:
X_train=X_train.reshape(-1,28,28,1)

In [ ]:
X_train.shape

(60000, 28, 28, 1)

In [ ]:
def display_images():
  noise = np.random.normal(0, 1, size=(samples, NOISE_DIM))
  generated_images = generator.predict(noise)
  plt.figure(figsize=(10, 10))
  for i in range(samples):
    plt.subplot(5,5, i+1)
    plt.imshow(generated_images[i].reshape(28, 28), cmap='binary')
    plt.axis('off')
  plt.show()

In [ ]:
## Training Loop

d_losses = []
g_losses = []

for epoch in range(TOTAL_EPOCHS):

    epoch_d_loss = 0.0
    epoch_g_loss = 0.0

    # Mini batch gradient descent
    for step in range(NO_OF_BATCHES):

      #step 1 Train Discriminator
      discriminator.trainable = True

      #get the real data
      idx = np.random.randint(0, 60000, HALF_BATCH)
      real_imgs = X_train[idx]

      # get the fake data
      noise = np.random.normal(0, 1, size=(HALF_BATCH, NOISE_DIM))
      fake_imgs = generator.predict(noise)

      # Labels
      real_y = np.ones((HALF_BATCH, 1)) * 0.9
      fake_y = np.zeros((HALF_BATCH, 1))

      #now train D

      d_loss_real = discriminator.train_on_batch(real_imgs, real_y)
      d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_y)

      d_loss = 0.5 * (d_loss_real + d_loss_fake)

      epoch_d_loss += d_loss

      #step 2 Train Generator
      discriminator.trainable = False # Freeze discriminator weights during generator training

      noise = np.random.normal(0, 1, size=(BATCH_SIZE, NOISE_DIM))
      ground_truth_y = np.ones((BATCH_SIZE, 1))

      g_loss = model.train_on_batch(noise, ground_truth_y)

      epoch_g_loss += g_loss

    print(f"Epoch{epoch+1}, Disc loss {epoch_d_loss/ NO_OF_BATCHES}, Generator loss {epoch_g_loss/ NO_OF_BATCHES}")

    d_losses.append(epoch_d_loss/ NO_OF_BATCHES)
    g_losses.append(epoch_g_loss/ NO_OF_BATCHES)

    if(epoch+1) % 10 == 0:
      generator.save(f"generator.h5")
      # display_images() # This function is not defined, so I'll keep it commented out for now.

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
4/4 ━━━━━━━━

KeyboardInterrupt: 